# Prueba a

In [1]:
import os
import pickle
from graphs.deep_queries_processes_agent import deep_queries_processes_agent
from graphs.myPrompts import SYSTEM_DEEP_QUERIES_PROMPT_1, HUMAN_DEEP_QUERIES_PROMPT_1
from langchain_core.messages import SystemMessage, HumanMessage


In [2]:
path_ditct = r"C:\Users\Acer\Documents\dev\mine-sql\.db_docs\sql-miner"
file_name = "mydict.pkl"
file = os.path.join(path_ditct, file_name)
with open(file, "rb") as f:
    mydict = pickle.load(f)

summary = mydict["summary"]
print(summary)
print("\n\n")


topic = mydict["topic"][0]
print(topic)
print("\n\n")


path_file = r"C:\Users\Acer\Documents\dev\mine-sql\.db_docs\sql-miner\queries.txt"
with open(file=path_file, encoding="utf-8") as f:
    querys = f.read()


Esta base de datos está diseñada para gestionar y analizar información relacionada con proyectos de inversión, particularmente en el sector de energías renovables y agricultura a pequeña escala (minifarms). El esquema incluye tablas para gestionar la información geográfica (países, regiones, ciudades, barrios), la autenticación y autorización de usuarios, la gestión de documentos (termsheets, contratos, informes), el seguimiento de la actividad del usuario (logs, auditoría), la gestión de tareas asíncronas (Celery Beat), la gestión de la comunicación (mensajes de WhatsApp, recordatorios), y la evaluación de terrenos. Un aspecto clave es la integración con servicios externos como ClickUp, Discord, Odoo, ZapSign y Facebook Graph API. La base de datos permite un seguimiento detallado del ciclo de vida de los proyectos, desde la prospección inicial y la evaluación del terreno hasta la firma de contratos y la gestión de pagos.



Gestión de Proyectos de Inversión: Permite el seguimiento del

In [3]:
system_prompt = SYSTEM_DEEP_QUERIES_PROMPT_1.format(summary=summary, topic=topic, querys=querys)
human_prompt = HUMAN_DEEP_QUERIES_PROMPT_1.format(topic=topic)

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=human_prompt)
]

initial_state = {"messages":messages}

In [ ]:
response = deep_queries_processes_agent.invoke(
    initial_state, 
    {"recursion_limit": 100}  # Aumenta el límite según necesidad
)

xxxxxxxxxxxxxxx initial_node
xxxxxxxxxxxxxxx
xxxxxxxxxxxxxxx


query inicial:

================================== Ai Message ==================================

```json
{
  "query": "SELECT DISTINCT stage FROM minifarm_project;",
  "processes": null
}
```











xxxxxxxxxxxxxxx set_ReAct_messages_node
xxxxxxxxxxxxxxx
xxxxxxxxxxxxxxx


len messages_react:2











xxxxxxxxxxxxxxx ReAct_node
xxxxxxxxxxxxxxx
xxxxxxxxxxxxxxx


type response_query: <class 'str'>



Current Message ReAct:

================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (5b9a8def-3b87-4702-a708-520344c8ed3e)
 Call ID: 5b9a8def-3b87-4702-a708-520344c8ed3e
  Args:
    tool_input: {}











--------------- should_continue_with_sql
---------------



tool_node_wrapper











xxxxxxxxxxxxxxx tool_node_wrapper
xxxxxxxxxxxxxxx
xxxxxxxxxxxxxxx


<class 'dict'>



{'messages_react': [ToolMessage(content='Error: 1 validation error for _ListSQLDatabaseTool

# Prueba b


================================== Ai Message ==================================

```json
{
  "query" : "¿Cuántos proyectos hay en cada etapa del proceso de inversión? (Consultar: minifarm_project, donde se indica la columna que representa la etapa, por ejemplo: status o stage)",
  "processes" : null
}
```
================================ Human Message =================================

Los proyectos se distribuyen en las distintas etapas del proceso de inversión de la siguiente forma:

| Etapa del proyecto | Número de proyectos |
|---------------------|---------------------|
| **dead**            | 1 825 |
| **prospect**        | 133 |
| **portfolio**       | 117 |
| **paused**          | 108 |
| **due_diligence**   | 106 |
| **negociation**     | 93 |
| **signed**          | 56 |
| **deploy**          | 23 |
| **operation**       | 20 |
| **construction**    | 20 |

Estas cifras provienen de la tabla `minifarm_project`, agrupando por la columna `stage` que indica la etapa en el proceso de inversión.
================================== Ai Message ==================================

```json
{
  "query" : "¿Cuántos proyectos están en la etapa 'due_diligence' y su última actualización (updated_at) es hace más de 60 días? (Consultar: minifarm_project, donde stage = 'due_diligence' y updated_at < NOW() - INTERVAL '60 days')",
  "processes" : null
}
```
================================ Human Message =================================

Hay **45** proyectos en la tabla `minifarm_project` que están en la etapa `'due_diligence'` y cuya última actualización (`updated_at`) fue hace más de 60 días.
================================== Ai Message ==================================

```json
{
  "query" : "Listar los 45 proyectos en la etapa 'due_diligence' con su ID, título, fecha de última actualización, el usuario responsable (owner_id) y su nombre completo, además de indicar si existen tareas asociadas en ClickUp (cálculo de count de tareas). (Consultar: minifarm_project, auth_user, monitoring_clickuptask, con LEFT JOIN y WHERE stage='due_diligence' AND updated_at < NOW() - INTERVAL '60 days')",
  "processes" : null
}
```
================================ Human Message =================================

**Observaciones previas**

* En la tabla **`minifarm_project`** no existe una columna llamada `owner_id`.
  (La única referencia a “owner” que aparece en el esquema es la columna `spcowner` de la tabla interna `pg_tablespace`, que no está relacionada con los proyectos).

* Dado que no hay una relación directa entre el proyecto y el usuario responsable, la única información que podemos devolver sobre el propietario es la que sí exista en la base de datos.
  Si en el futuro se crea una columna `owner_id` (o una tabla de relación) se podrá incluir esa información.

* Para contar las tareas de ClickUp asociadas a cada proyecto, utilizamos la columna `clickup_task_id` de la tabla `minifarm_project`.
  En la muestra de datos proporcionada esa columna está vacía, por lo que el conteo será 0.
  Si se dispone de datos reales de `clickup_task_id`, el conteo reflejará correctamente el número de tareas.

---

## Consulta que devuelve los 45 proyectos más antiguos en la etapa `due_diligence`

```sql
-- 1. Seleccionamos los campos requeridos
-- 2. Filtramos por la etapa y la fecha de actualización
-- 3. Contamos tareas de ClickUp asociadas al proyecto
-- 4. Limitamos a 45 filas
SELECT
    p.id,
    p.name,
    p.updated_at,
    -- Información del propietario (si existe una columna owner_id en el futuro)
    -- u.id      AS owner_id,
    -- u.first_name || ' ' || u.last_name AS owner_name,
    -- En la actualidad no hay columna owner_id, así que lo dejamos comentado
    COALESCE(
        (SELECT COUNT(*) FROM monitoring_clickuptask t
         WHERE t.id = p.clickup_task_id),
        0
    ) AS task_count
FROM
    minifarm_project p
-- LEFT JOIN auth_user u ON u.id = p.owner_id   -- Descomentar cuando exista la columna
WHERE
    p.stage = 'due_diligence'
    AND p.updated_at < NOW() - INTERVAL '60 days'
ORDER BY
    p.updated_at ASC            -- Los más antiguos primero
LIMIT 45;
```

---

## Resultado (con la muestra de datos que tenemos)

| id     | name                               | updated_at           | task_count |
|--------|------------------------------------|----------------------|------------|
| 45737  | COLTOLT181P1_SAN-LUIS_SUR          | 2025-07-11 08:42:20  | 0          |
| 45961  | COLMAGT216P1_PIVIJAY_SUR           | 2025-09-03 17:32:37  | 0          |
| 45071  | COLSANT100P1_CIMITARRA_NORTE       | 2025-09-04 09:36:43  | 0          |

> **Nota**: Los valores de `task_count` son 0 porque la columna `clickup_task_id` está vacía en la muestra.
> En un entorno real, si se cargan valores válidos, el conteo cambiará a la cantidad correcta de tareas asociadas a cada proyecto.

---

### Próximos pasos

1. **Añadir la columna `owner_id`** (o tabla de relación) a `minifarm_project` si se desea enlazar el proyecto con el usuario responsable.
2. **Poblar la columna `clickup_task_id`** con los identificadores reales de ClickUp.
3. **Re‑ejecutar la consulta** para obtener la información completa: proyecto, responsable y número de tareas.

================================== Ai Message ==================================

```json
{
  "query" : "¿Cuáles son todas las columnas que existen en la tabla `minifarm_project`?",
  "processes" : null
}
```
================================ Human Message =================================

**Columnas de la tabla `minifarm_project`**

| Número | Columna |
|--------|---------|
| 1 | id |
| 2 | name |
| 3 | clickup_task_id |
| 4 | project_type |
| 5 | circuit_id |
| 6 | lat |
| 7 | lng |
| 8 | clickup_list_id |
| 9 | phase_id |
| 10 | supplies_status_external |
| 11 | supplies_status_internal_id |
| 12 | terrain_id |
| 13 | city_prev_id |
| 14 | stage |
| 15 | area_m2 |
| 16 | unergy_topic_name |
| 17 | contract_type |
| 18 | grid_operator_id |
| 19 | annual_price |
| 20 | network_distance |
| 21 | road_distance |
| 22 | project_extension |
| 23 | project_installed_power |
| 24 | project_panels_description |
| 25 | created_at |
| 26 | updated_at |
| 27 | drive |
| 28 | termsheet_id |
| 29 | city_id |
| 30 | odoo_analytic_account_id |
| 31 | banner |
| 32 | project_panels_count |

Estas son todas las columnas que existen en la tabla `minifarm_project`.

================================== Ai Message ==================================

```json
{
  "query" : "Obtener las entradas del log de administración que modifican el campo `stage` en la tabla `minifarm_project` en los últimos 30 días, mostrando el ID del proyecto, el valor anterior, el valor nuevo, el usuario que hizo el cambio y la fecha del cambio. (Consultar: django_admin_log, auth_user, content_type; filtrar por content_type_id correspondiente a `minifarm_project` y que change_message contenga 'stage')",
  "processes" : null
}
```
================================ Human Message =================================

I’m ready to help with your database query!
What information would you like to retrieve or analyze from the database? Let me know the question or the specific data you’re interested in, and I’ll craft the appropriate SQL query for you.

================================== Ai Message ==================================

```json
{
  "query" : "Seleccionar todos los proyectos que han quedado en la misma etapa durante más de 45 días: obtener id, nombre, etapa actual, fecha de la última actualización y cuántos días han pasado desde entonces. (Consultar: minifarm_project, calcular edad con NOW() - updated_at, filtrar por age > INTERVAL '45 days' y stage NOT IN ('dead','operation'))",
  "processes" : null
}
```
================================ Human Message =================================

**Proyectos que han permanecido en la misma etapa por más de 45 días (última actualización)**

| ID  | Nombre del proyecto                      | Etapa actual | Última actualización (updated_at) | Días transcurridos desde la última actualización |
|-----|------------------------------------------|--------------|------------------------------------|-------------------------------------------------|
| 45236 | COLCUNT139P1_TOCANCIPA_NORTE | paused | 2025‑04‑15 13:26:13 | 190 |
| 45234 | COLCUNT137P1_GUASCA_OCCIDENTE | paused | 2025‑04‑15 13:26:12 | 190 |
| 45235 | COLCUNT138P1_LENGUAZAQUE_NORTE | paused | 2025‑04‑15 13:26:13 | 190 |
| 45106 | COLRIST4P2_PEREIRA_OCCIDENTE | paused | 2025‑04‑15 13:26:09 | 190 |
| 12500 | COLATLT56P1_REPELON_SUR | paused | 2025‑04‑15 13:26:10 | 190 |
| 4374  | COLARAT1P2_ARAUCA_OCCIDENTE | paused | 2025‑04‑15 13:29:26 | 190 |
| 2505  | COLCEST7P1_LA‑PAZ_NORTE | portfolio | 2025‑04‑15 13:26:55 | 190 |
| 12501 | COLNORT19P1_TIBU_ORIENTE | negociation | 2025‑04‑15 13:26:11 | 190 |
| 11114 | COLCEST7P2_LA‑PAZ_NORTE | portfolio | 2025‑04‑15 13:26:55 | 190 |
| 45239 | COLCUNT142P1_TENJO_OCCIDENTE | paused | 2025‑04‑15 13:26:15 | 190 |

> **Nota:**
> - La columna *“Días transcurridos”* se calcula como la diferencia entre la fecha actual (`NOW()`) y el campo `updated_at`.
> - Se excluyeron las etapas `dead` y `operation`, tal como se indicó.
> - El resultado se limitó a 10 filas y se ordenó descendente por la antigüedad de la última actualización.
================================== Ai Message ==================================

```json
{
  "query" : "¿Cuántos de los proyectos que han estado en la misma etapa durante más de 45 días (según la tabla `minifarm_project` y el campo `updated_at`) tienen al menos una tarea asignada en ClickUp? (Consultar: minifarm_project LEFT JOIN monitoring_clickuptask ON monitoring_clickuptask.id = minifarm_project.clickup_task_id, filtrar por NOW() - minifarm_project.updated_at > INTERVAL '45 days', y contar distintamente los proyectos con count(t.id) > 0)",
  "processes" : null
}
```

